In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import lxml.html
import re

import pandas as pd
from pandas.io.json import json_normalize

import datetime
import time

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
url = 'https://www.op.gg/summoner/champions/userName='
# api_key = 'RGAPI-bee4c76f-7883-43ca-a8ae-45131dbd1cc1'
# api = '?api_key='+api_key

# 1. 소환사명 불러오기

Challenger to Diamond2

In [309]:
ch_to_dia_df = pd.read_csv('./data/ch_to_dia.csv')
ch_to_dia_df.drop(columns=['Unnamed: 0'], inplace=True)
ch_to_dia_df.head()

,summonerName,summonerId,tier,rank,wins,losses,leagueId,leaguePoints,queueType,veteran,freshBlood,hotStreak,inactive,miniSeries.wins,miniSeries.losses,miniSeries.progress,miniSeries.target
0,김도칠,1632YZc0zcIwC5zX3fXYHhFRXycDtgN6XhWf4Ttf2w6VoAc,NaN,I,410,365,NaN,837,NaN,False,False,False,False,NaN,NaN,NaN,NaN
1,DWG BeryL,9leRGjqDnEqVt9QBtKOUM2cIdDOuV4aUmlnvF5olrtUz5fY,NaN,I,298,258,NaN,875,NaN,True,False,False,False,NaN,NaN,NaN,NaN
2,실망자,cdya4bpOdk15ISCADNsLWs0GolBDkHJ5-zgxU-wsJz0zCY0,NaN,I,319,281,NaN,774,NaN,True,False,False,False,NaN,NaN,NaN,NaN
3,랭겜에강한남자,0NZwD5ownbTJ1EDYTcKgWJgS04MIBvN8Q2gvA7sOsTFFrY0,NaN,I,375,323,NaN,719,NaN,False,False,False,False,NaN,NaN,NaN,NaN
4,sheng huo bu yi,o3HTID39FWbQSx9UARP51aTANi7-gYLw9StO5ntIXqzFbQ,NaN,I,358,313,NaN,789,NaN,False,False,False,False,NaN,NaN,NaN,NaN


# 2. 소환사별 챔피언 정보 정리하기

In [233]:
regex = {}

regex['Champion'] = re.compile(r'\s*([\w.&]*)\s*')
regex['Played'] = re.compile(r'\s*(\d*W)*\s*(\d*L)*\s*(\d*%)+\s*')
regex['KDA'] = re.compile(r'\s*([\d.]*)[ /\s]*([\d.]*)[ /\s]*([\d.]*)[ /\s]*([\d.:]*)\s*')
regex['Gold'] = re.compile(r'\s*([\d,]*)\s*')
regex['CS'] = re.compile(r'\s*([\d.]*)[\(\s]*([\d.]*)\)\s*')
regex['Max Kills'] = re.compile(r'\s*([\d]*)\s*')
regex['Max Death'] = re.compile(r'\s*([\d]*)\s*')
regex['Average Damage Dealt'] = re.compile(r'\s*([\d,]*)\s*')
regex['Average Damage Taken'] = re.compile(r'\s*([\d,]*)\s*')
regex['Double Kill'] = re.compile(r'\s*([\d]*)\s*')
regex['Triple Kill'] = re.compile(r'\s*([\d]*)\s*')
regex['Quadra Kill'] = re.compile(r'\s*([\d]*)\s*')
regex['Penta Kill'] = re.compile(r'\s*([\d]*)\s*')

## opgg에서 소환사명으로 소환사별 챔피언 데이터 불러오기

In [ ]:
columnlist = ['#', 'Champion', 'Win', 'Loss', 'Win Rate', 'K', 'D', 'A', 'KDA', 'Gold', 'CS', 'CS/min', 
               'Max Kills', 'Max Deaths', 'Average Damage Dealt', 'Average Damage Taken',
              'Double Kill', 'Triple Kill', 'Quadra Kill', 'Penta Kill']
DF = pd.DataFrame(columns=['summonerName']+columnlist[1:])

In [255]:
j = 4382
N = len(ch_to_dia_df.summonerName.values)

In [ ]:
while j < N:
    try:    
        sn = ch_to_dia_df.summonerName.values[j]
        sn_noblank = re.sub(r'[\s]*', '', sn)
        encode = quote_plus(sn_noblank)

        html = urlopen(url+encode)
        bsObject = BeautifulSoup(html, 'html.parser')
        contents = bsObject.select('div > table > tbody > tr')

        reg = re.compile(r'[\d\w.,:&\' ]+')

        df = pd.DataFrame(columns=columnlist)
        dfcontent = []

        for con in contents:
            tds = con.find_all('td')
            del tds[1]

            for i, td in enumerate(tds):
                text = td.text
                texts = reg.findall(text)

                if i == 2:
                    played_group = regex['Played'].search(text)

                    if played_group.group(1) == None:
                        dfcontent = dfcontent + ['0W'] + texts
                    elif played_group.group(2) == None:
                        dfcontent = dfcontent + [texts[0]] + ['0L'] + [texts[1]]
                    else:
                        dfcontent = dfcontent + texts

                else:
        #         dfcontent.append(text)
                    dfcontent = dfcontent + texts

            if len(dfcontent) != len(columnlist):
                for i in range(len(columnlist)-len(dfcontent)):
                    dfcontent.append(0)

            ser = pd.Series(dfcontent, index=columnlist)
            df = df.append(ser, ignore_index=True)

            dfcontent = []

        # df = pd.DataFrame()
        # df = df.append(alldfcontents)
        df['Win'] = df['Win'].apply(lambda x: x[:-1])
        df['Loss'] = df['Loss'].apply(lambda x: x[:-1])
    
    except Exception as e:
        print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, e))
        continue

    df.drop(columns=['#'], inplace=True)
    df['summonerName'] = sn
    DF = DF.append(df)
    
    print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), j, sn))
    j += 1
    time.sleep(1)

In [257]:
DF.to_csv('./data/opgg_ch_to_dia.csv')

## 챔피언 데이터 정리하기

In [258]:
opgg_df = pd.read_csv('./data/opgg_ch_to_dia.csv')
opgg_df.drop(columns=['Unnamed: 0'], inplace=True)
opgg_df = opgg_df[['summonerName', 'Champion', 'Win', 'Loss', 'Win Rate', 'KDA', 'K', 'D', 'A', 
                  'CS', 'CS/min', 'Gold', 'Max Kills', 'Max Deaths', 'Average Damage Dealt', 'Average Damage Taken', 
                   'Double Kill', 'Triple Kill', 'Quadra Kill', 'Penta Kill']]
opgg_df.head()

C:\Users\LG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,summonerName,Champion,Win,Loss,Win Rate,KDA,K,D,A,CS,CS/min,Gold,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill
0,Hide on bush,LeBlanc,38,29,57,4.15:1,6.5,3.0,6.1,196.4,7.6,"11,027",22,7,"105,577","17,692",43,8,0,0
1,Hide on bush,Sett,45,20,69,3.91:1,4.7,2.9,6.5,190.9,7.4,"10,872",13,7,"114,469","24,264",29,3,0,0
2,Hide on bush,Sylas,27,35,44,2.68:1,5.9,4.5,6.2,211.1,7.7,"11,784",16,11,"131,697","26,902",39,5,1,0
3,Hide on bush,Zoe,32,29,52,4.35:1,6.0,3.0,7.3,212.5,7.7,"11,884",19,8,"140,932","13,991",31,3,1,0
4,Hide on bush,Lucian,28,18,61,2.61:1,4.9,3.9,5.3,226.2,8.7,"11,525",13,8,"143,392","16,434",21,2,1,0


In [259]:
opgg_df['Gold'] = opgg_df['Gold'].apply(lambda x: int(x.replace(',', '')))
opgg_df['Average Damage Dealt'] = opgg_df['Average Damage Dealt'].apply(lambda x: int(x.replace(',', '')))
opgg_df['Average Damage Taken'] = opgg_df['Average Damage Taken'].apply(lambda x: int(x.replace(',', '')))
opgg_df['Double Kill'] = opgg_df['Double Kill'].apply(lambda x: str(x))
opgg_df['Double Kill'] = opgg_df['Double Kill'].apply(lambda x: int(x.replace(',', '')))
def cleanse_kda(x):
    if x == 'Perfect':
        return 100.0
    else:
        return float(x[:x.find(':')])
    
opgg_df['KDA'] = opgg_df['KDA'].apply(lambda x: cleanse_kda(x))
opgg_df['Games'] = opgg_df['Win'] + opgg_df['Loss']

In [260]:
for col in opgg_df.columns:
    print(col, opgg_df[col].dtype)

summonerName object
Champion object
Win int64
Loss int64
Win Rate int64
KDA float64
K float64
D float64
A float64
CS float64
CS/min float64
Gold int64
Max Kills int64
Max Deaths int64
Average Damage Dealt int64
Average Damage Taken int64
Double Kill int64
Triple Kill int64
Quadra Kill int64
Penta Kill int64
Games int64


In [261]:
opgg_df = opgg_df[['summonerName', 'Champion', 'Win', 'Loss', 'Games', 'Win Rate', 'KDA', 'K', 'D', 'A', 
                  'CS', 'CS/min', 'Gold', 'Max Kills', 'Max Deaths', 'Average Damage Dealt', 'Average Damage Taken', 
                   'Double Kill', 'Triple Kill', 'Quadra Kill', 'Penta Kill']]
opgg_df.to_csv('./data/opgg_ch_to_dia_cleansed.csv')

# 3. 챔피언 ID 매칭하기

In [262]:
opgg_df = pd.read_csv('./data/opgg_ch_to_dia_cleansed.csv')
opgg_df.drop(columns=['Unnamed: 0'], inplace=True)
opgg_df.head()

,summonerName,Champion,Win,Loss,Games,Win Rate,KDA,K,D,A,...,CS/min,Gold,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill
0,Hide on bush,LeBlanc,38,29,67,57,4.15,6.5,3.0,6.1,...,7.6,11027,22,7,105577,17692,43,8,0,0
1,Hide on bush,Sett,45,20,65,69,3.91,4.7,2.9,6.5,...,7.4,10872,13,7,114469,24264,29,3,0,0
2,Hide on bush,Sylas,27,35,62,44,2.68,5.9,4.5,6.2,...,7.7,11784,16,11,131697,26902,39,5,1,0
3,Hide on bush,Zoe,32,29,61,52,4.35,6.0,3.0,7.3,...,7.7,11884,19,8,140932,13991,31,3,1,0
4,Hide on bush,Lucian,28,18,46,61,2.61,4.9,3.9,5.3,...,8.7,11525,13,8,143392,16434,21,2,1,0


In [263]:
champ_key_df = pd.read_csv('./data/champ_key_10.15.1.csv')
champ_key_df.rename(columns={'name':'Champion',
                            'key':'championId'},
                   inplace=True)
champ_key_df.head()

,championId,Champion
0,1,Annie
1,2,Olaf
2,3,Galio
3,4,Twisted Fate
4,5,Xin Zhao


In [264]:
opgg_df = pd.merge(opgg_df, champ_key_df, on='Champion', how='left')
opgg_df.to_csv('./data/opgg_ch_to_dia_cleansed.csv')
opgg_df.head()

,summonerName,Champion,Win,Loss,Games,Win Rate,KDA,K,D,A,...,Gold,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill,championId
0,Hide on bush,LeBlanc,38,29,67,57,4.15,6.5,3.0,6.1,...,11027,22,7,105577,17692,43,8,0,0,7
1,Hide on bush,Sett,45,20,65,69,3.91,4.7,2.9,6.5,...,10872,13,7,114469,24264,29,3,0,0,875
2,Hide on bush,Sylas,27,35,62,44,2.68,5.9,4.5,6.2,...,11784,16,11,131697,26902,39,5,1,0,517
3,Hide on bush,Zoe,32,29,61,52,4.35,6.0,3.0,7.3,...,11884,19,8,140932,13991,31,3,1,0,142
4,Hide on bush,Lucian,28,18,46,61,2.61,4.9,3.9,5.3,...,11525,13,8,143392,16434,21,2,1,0,236


# 4. MATCH 데이터와 소환사 데이터 merge 하기

## match 데이터 불러오기

In [353]:
matches_df = pd.read_csv('./data/matches.csv')
matches_df.drop(columns=['Unnamed: 0'], inplace=True)

print('전체 데이터:         ', len(matches_df))

전체 데이터:          39198


## 소환사의 협곡 데이터만 걸러내기

In [354]:
classic_matches_df = matches_df.loc[matches_df['gameMode'] == 'CLASSIC']
print('소환사의 협곡 데이터: ', len(classic_matches_df))

classic_matches_df.tail()

소환사의 협곡 데이터:  32152


,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,platformId,queueId,seasonId,...,team2_4_lane,team2_4_role,team2_5_championId,team2_5_spell1Id,team2_5_spell2Id,team2_5_accountId,team2_5_summonerName,team2_5_summonerId,team2_5_lane,team2_5_role
39192,1594701435931,969,4507648967,CLASSIC,MATCHED_GAME,10.14.327.8505,11,KR,420,13,...,NONE,DUO_SUPPORT,20,4,11,4ait8wbOM5SPQ4Z3hQoDv207A2GUjw7PsypEpvvbgZS_vx...,원시챔피언,Za4-mUqbO25dFgPUTCjWEKtK55txFzK-NMqU-0UU4h4I90Q,NONE,DUO_SUPPORT
39193,1592749423513,1947,4462560202,CLASSIC,MATCHED_GAME,10.12.325.6222,11,KR,430,13,...,JUNGLE,NONE,516,4,12,IJLUK0LkcoxnIowzX8KjQ3mSSeb1d0pNYG4rXXPLXSDw27...,나는 우주먼지야,LfSy_EDHCL7L0S653cGKb0H6_DzTcaugZE9hTG4rdBFHC2g,MIDDLE,DUO
39194,1595990755183,936,4538581967,CLASSIC,MATCHED_GAME,10.15.330.344,11,KR,430,13,...,NONE,DUO_SUPPORT,497,14,4,rozj7nYD2rxa82Ex1sZUMyuNe8OfDtetWoFmOhFTNoKP,알아보고칠하자,epqjQ6_va5v3ji6k_2rxeraVCnh9jSi5HUVLnnP-z4AgtQ,NONE,DUO_SUPPORT
39195,1594564134901,1835,4505027537,CLASSIC,MATCHED_GAME,10.14.327.8505,11,KR,430,13,...,TOP,SOLO,523,7,4,CdHgVHTzn072Bu5rFnQpXOUIjFMPpyup4mGcLe4ATjNja1...,학일계군,vsi8_99Gr7bv5R2NAA0R4OoUUhG3E3pMtArtUrY51nYEkmQ,BOTTOM,DUO_CARRY
39196,1595934801542,1362,4537009107,CLASSIC,MATCHED_GAME,10.15.328.8759,11,KR,420,13,...,MIDDLE,DUO_SUPPORT,22,1,4,fk1P8Q7CxiaOIVvVe-lWEpDwoXgY3h8y5p-iab8G5FlJ,하이드,Qatu2ZUi7z9eHptUZidXeHZizEn-h6ZQAkm8fhB1YcDqVw,MIDDLE,DUO


## 소환사 ID로 소환사 이름 새로 매칭하기

닉네임 작작 좀 바꿔...

In [364]:
for col in classic_matches_df.columns:
    if 'team1_1' in col:
        print(col)
print()


# 소환사 목록 불러오기

ch_to_dia_df = pd.read_csv('./data/ch_to_dia.csv')
ch_to_dia_df.drop(columns=['Unnamed: 0'], inplace=True)
print('challenger to dia2: ', len(ch_to_dia_df))
ch_to_dia_df.head()

team1_1_championId
team1_1_spell1Id
team1_1_spell2Id
team1_1_accountId
team1_1_summonerName
team1_1_summonerId
team1_1_lane
team1_1_role
team1_1_key

challenger to dia2:  22203


,summonerName,summonerId,tier,rank,wins,losses,leagueId,leaguePoints,queueType,veteran,freshBlood,hotStreak,inactive,miniSeries.wins,miniSeries.losses,miniSeries.progress,miniSeries.target
0,Hide on bush,04E81p_9PA-iPR8e4m1ZsU0Jo8ZsjDNJXJf-KZ3q1dg8Gg,NaN,I,398,356,NaN,908,NaN,True,False,False,False,NaN,NaN,NaN,NaN
1,DWG BeryL,9leRGjqDnEqVt9QBtKOUM2cIdDOuV4aUmlnvF5olrtUz5fY,NaN,I,298,258,NaN,875,NaN,True,False,False,False,NaN,NaN,NaN,NaN
2,실망자,cdya4bpOdk15ISCADNsLWs0GolBDkHJ5-zgxU-wsJz0zCY0,NaN,I,319,281,NaN,774,NaN,True,False,False,False,NaN,NaN,NaN,NaN
3,랭겜에강한남자,0NZwD5ownbTJ1EDYTcKgWJgS04MIBvN8Q2gvA7sOsTFFrY0,NaN,I,375,323,NaN,719,NaN,False,False,False,False,NaN,NaN,NaN,NaN
4,sheng huo bu yi,o3HTID39FWbQSx9UARP51aTANi7-gYLw9StO5ntIXqzFbQ,NaN,I,358,313,NaN,789,NaN,False,False,False,False,NaN,NaN,NaN,NaN


In [365]:
# dictionary{ summonerId : summonerName } 만들기

summonerIds = ch_to_dia_df.summonerId.values
summonerNames = ch_to_dia_df.summonerName.values

summonerIds_dict = dict(zip(summonerIds, summonerNames))


# summonerId에 따라 summonerName 돌려주는 함수
def get_summonerName(summonerId):
    if summonerId in summonerIds_dict:
        return summonerIds_dict[summonerId]
    else:
        return None

# summonerId에 맞게 summonerName 수정
for team in [1,2]:
    for summoner in range(1,6):
        classic_matches_df['team{0}_{1}_summonerName'.format(team, summoner)] = \
                classic_matches_df['team{0}_{1}_summonerId'.format(team, summoner)].map(lambda x: get_summonerName(x))
        
        
# nan 값 처리
classic_matches_df.dropna(inplace=True)


# summonerName:championId 형태로 key 설정
for team in [1,2]:
    for summoner in range(1,6):
        classic_matches_df['team{0}_{1}_key'.format(team, summoner)] = classic_matches_df['team{0}_{1}_summonerName'.format(team, summoner)] + \
        ':' +  classic_matches_df['team{0}_{1}_championId'.format(team, summoner)].apply(lambda x: str(x))
        
        
print('데이터 수: ', len(classic_matches_df))
classic_matches_df.head()
classic_matches_df.to_csv('./data/classic_matches.csv')

데이터 수:  19794


## match 데이터와 소환사별 챔피언 데이터 합치기

In [367]:
opgg_df = pd.read_csv('./data/opgg_ch_to_dia_cleansed.csv')
opgg_df.drop(columns=['Unnamed: 0'], inplace=True)


# 소환사이름:챔피언아이디 로 key 설정

opgg_df['key'] = opgg_df['summonerName'] + ':' + opgg_df['championId'].apply(lambda x: str(x))
opgg_df.head()

,summonerName,Champion,Win,Loss,Games,Win Rate,KDA,K,D,A,...,Max Kills,Max Deaths,Average Damage Dealt,Average Damage Taken,Double Kill,Triple Kill,Quadra Kill,Penta Kill,championId,key
0,Hide on bush,LeBlanc,38,29,67,57,4.15,6.5,3.0,6.1,...,22,7,105577,17692,43,8,0,0,7,Hide on bush:7
1,Hide on bush,Sett,45,20,65,69,3.91,4.7,2.9,6.5,...,13,7,114469,24264,29,3,0,0,875,Hide on bush:875
2,Hide on bush,Sylas,27,35,62,44,2.68,5.9,4.5,6.2,...,16,11,131697,26902,39,5,1,0,517,Hide on bush:517
3,Hide on bush,Zoe,32,29,61,52,4.35,6.0,3.0,7.3,...,19,8,140932,13991,31,3,1,0,142,Hide on bush:142
4,Hide on bush,Lucian,28,18,46,61,2.61,4.9,3.9,5.3,...,13,8,143392,16434,21,2,1,0,236,Hide on bush:236


In [370]:
classic_matches_df = pd.read_csv('./data/classic_matches.csv')
classic_matches_df.drop(columns=['Unnamed: 0'], inplace=True)
classic_matches_df.head()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,platformId,queueId,seasonId,...,team1_1_key,team1_2_key,team1_3_key,team1_4_key,team1_5_key,team2_1_key,team2_2_key,team2_3_key,team2_4_key,team2_5_key
0,1590930683807,966,4419747843,CLASSIC,MATCHED_GAME,10.11.322.2991,11,KR,420,13,...,onlyfunnygame:81,준승이:555,마이티베어:76,edgeeee:39,깡충깡충껑충껑충:57,병권좌:141,Mango Fish:105,초보들의 왕:145,땅우양유튜브구독:412,Platypus1:82
1,1592748605609,1605,4462477317,CLASSIC,MATCHED_GAME,10.12.325.6222,11,KR,420,13,...,fancy Kurumi:43,앳원 온 타이탄:104,Gen G 무:110,qinyu:57,wohennangu0:518,chixiaoyubaby:136,갑각류맛있댜:58,스위트 패스츄리:235,KKHHHK:223,뭐하는디야:48
2,1591035247840,1890,4422369287,CLASSIC,MATCHED_GAME,10.11.322.2991,11,KR,420,13,...,iudzhk:76,팬리구너:62,sdasf dfghj:110,zhendeshuang:412,Kiirito:3,DRX 홍창현:28,edgeeee:236,독한놈1:53,BLG ADD:875,바이올렛에버 :145
3,1596010510872,1314,4539023370,CLASSIC,MATCHED_GAME,10.15.330.344,11,KR,420,13,...,종탁이:99,BIank Space:51,ke ke ai ai:60,섬광의 무희 애디:3,손민형씨올라프:246,hxzismyb:104,xxxxxxx1:555,Re0start:61,우싹씨:164,hy2rwt:134
4,1592975909614,1420,4467195917,CLASSIC,MATCHED_GAME,10.13.325.7485,11,KR,420,13,...,huntiand1:164,Iove galio:3,호잇이:350,승부욕xxx:104,SANDBOX Leo:81,owoowoowoowoowo:245,asdade:421,Da Su Ke De:119,HLE Mireu:266,Miunaovo:412


In [371]:
summoner_champion_properties = ['Games', 'Win Rate', 'KDA']

for team in [1,2]:
    for summoner in range(1,6):
        temp_opgg_df = opgg_df[['key'] + summoner_champion_properties]
        temp_opgg_df.columns = temp_opgg_df.columns.map(lambda x: 'team{0}_{1}_'.format(team, summoner) + x)

        classic_matches_df = pd.merge(classic_matches_df, temp_opgg_df, 
                              left_on='team{0}_{1}_key'.format(team, summoner), 
                              right_on='team{0}_{1}_key'.format(team, summoner), 
                              how='left')
        
# nan 값 처리
classic_matches_df.dropna(inplace=True)
print('데이터 수: ', len(classic_matches_df))
classic_matches_df.to_csv('./data/matches_for_train.csv')

데이터 수:  15331


In [372]:
classic_matches_df.head()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,platformId,queueId,seasonId,...,team2_2_KDA,team2_3_Games,team2_3_Win Rate,team2_3_KDA,team2_4_Games,team2_4_Win Rate,team2_4_KDA,team2_5_Games,team2_5_Win Rate,team2_5_KDA
0,1590930683807,966,4419747843,CLASSIC,MATCHED_GAME,10.11.322.2991,11,KR,420,13,...,2.99,68.0,49.0,3.16,666.0,55.0,3.09,31.0,45.0,1.64
4,1592975909614,1420,4467195917,CLASSIC,MATCHED_GAME,10.13.325.7485,11,KR,420,13,...,3.07,63.0,54.0,2.78,74.0,55.0,2.11,158.0,54.0,3.22
5,1595753272081,2338,4532469773,CLASSIC,MATCHED_GAME,10.15.328.8759,11,KR,420,13,...,3.71,825.0,52.0,3.10,32.0,56.0,2.60,199.0,62.0,3.30
6,1595093179666,1539,4516479000,CLASSIC,MATCHED_GAME,10.14.327.8505,11,KR,420,13,...,3.04,11.0,55.0,2.50,106.0,60.0,3.03,10.0,60.0,3.13
8,1595259386597,1691,4520149020,CLASSIC,MATCHED_GAME,10.14.327.8505,11,KR,420,13,...,2.45,440.0,54.0,2.27,937.0,54.0,2.60,198.0,63.0,2.58
